In [41]:
%matplotlib inline
import numpy as np
import pandas as pd
import cvxpy as cp
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import genfromtxt
from scipy.optimize import linprog
plt.rcParams['figure.figsize'] = [10, 10]

### Data Generation
The underlying model is a linear model. I randomly generate weights $\beta$ for $p$ features. The underlying model is then $Y = X^T \beta + \epsilon$ where $\epsilon \sim N(0, 1)$

In [97]:
p = 10 # number of features
n = 100 # number of data points
beta = np.random.normal(0, 1, 10)
beta0 = np.random.normal(0, 1)
beta0, beta

(-0.792483307744766,
 array([ 2.31144497,  0.04354509,  0.20134108, -0.25421774,  1.77586718,
        -2.1133035 , -2.06072016, -0.58484004,  0.45857761,  0.70669958]))

In [98]:
X = np.random.normal(0, 1, (n, p))
Y = X @ beta + beta0*np.ones(n) + np.random.normal(0, 1, n)

### Regression Formula
Just using the formula $\hat{\beta} = (X^TX)^{-1}X^TY$ here

In [99]:
Xprime = np.insert(X, 0, np.ones(n), axis=1)
Xprime[0:5,:]

array([[ 1.        ,  0.40880543, -0.6827643 , -1.25341075,  0.35306643,
         1.43422226,  1.70509291,  1.84254312, -0.9478274 , -0.99488189,
        -0.32464899],
       [ 1.        ,  0.14765154,  1.33274188,  0.73230448, -1.35817493,
         0.18065067, -0.64459808, -0.4780891 ,  2.17016526,  0.54835863,
         0.47224033],
       [ 1.        ,  0.80253334, -0.2475641 ,  1.40368364, -1.25508336,
        -2.10943053,  0.29532366,  0.82362716,  0.64304665, -1.57034019,
        -2.96946787],
       [ 1.        ,  1.27543838,  0.11767872, -0.38042413, -1.78204858,
         0.60165393,  0.09501971,  0.32495483,  0.37054124,  0.81239681,
        -0.02820516],
       [ 1.        , -1.0460915 ,  0.15423387,  0.2765208 ,  0.25880292,
        -1.38462809, -0.02227528,  0.88678316, -0.15214923,  0.74950174,
        -0.29811004]])

In [100]:
betahat = np.linalg.inv(Xprime.T @ Xprime) @ Xprime.T @ Y
betahat

array([-0.67231775,  2.20987265,  0.09585375,  0.0556739 , -0.23511587,
        1.66939383, -2.05591397, -2.0634393 , -0.42606317,  0.38976968,
        0.76673522])

### Optimization
Solving it using CVXPY as an optimization problem i.e. not using exact form.

In [101]:
bhat = cp.Variable(p + 1)
cost = cp.sum_squares(Xprime @ bhat - Y)
prob = cp.Problem(cp.Minimize(cost))
prob.solve()

72.75110490292734

In [102]:
bhat.value

array([-0.67231775,  2.20987265,  0.09585375,  0.0556739 , -0.23511587,
        1.66939383, -2.05591397, -2.0634393 , -0.42606317,  0.38976968,
        0.76673522])

### Gram Schmidt and Projection
This method is basically projection $Y$ onto $X_i$ for $i = 1, ..., p$, eliminating the residue from $Y$ and repeating the process.

In [103]:
Yprime = Y.copy()
Yprime

array([-4.89052383e+00,  2.27956609e+00, -7.50516483e+00,  3.34712504e+00,
       -6.84678054e+00, -2.91254197e+00, -1.88290794e+00,  1.88310888e+00,
       -9.31228593e+00,  4.49593305e-02, -2.28526982e+00,  4.60590952e+00,
       -1.98663035e+00,  2.69204216e+00, -4.17170407e+00,  6.20001049e+00,
        1.04856957e+01, -4.51275045e+00, -4.97630065e+00, -3.71746395e+00,
        1.21858703e+00, -7.42319949e+00,  3.80913230e+00, -3.36826290e+00,
        5.46239902e+00, -2.61452218e+00, -6.30335923e-01,  2.98293744e-03,
       -3.86163431e+00, -5.09508851e-01,  2.25765374e+00,  4.24203515e+00,
        2.99892690e-01, -4.07888834e+00,  2.11457171e+00, -3.54888830e-01,
       -4.62596234e+00,  9.30428272e+00, -5.59877724e+00,  5.18534707e+00,
        6.38349824e+00, -3.98060547e+00,  2.83962032e+00, -1.64390393e+00,
       -2.58134545e+00,  3.71462040e+00, -7.20891829e+00,  6.01148980e+00,
       -4.03618862e+00, -1.54067855e+00, -2.62909291e+00, -1.79032321e+00,
       -3.85937112e-01, -

In [108]:
Xprime[:,0]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [105]:
res = []
bb = []
for i in range(p + 1):
    cur = Xprime[:, i].copy()
    for j in range(i):
        cur -= np.dot(cur, res[j])*res[j]
    curbeta = np.dot(Yprime, cur)/(np.linalg.norm(cur, 2)**2)
    Yprime -= curbeta*cur
    res.append(cur/np.linalg.norm(cur, 2))
    bb.append(curbeta)

In [106]:
bb

[-0.7487446312045952,
 2.145417786784691,
 0.10980869718655849,
 0.009680004291956609,
 -0.5561378640119993,
 2.13266580846764,
 -1.8533279111122443,
 -2.159478838117387,
 -0.4136379037716033,
 0.40190847124973944,
 0.7667352164662559]